In [66]:
# Importacion de las librerías necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE

In [2]:
# Cargar la data
df = pd.read_csv('Anexo 1 - Attrition.csv') 

In [60]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## Contextualizacion: 

El laboratorio de Analítica de IBM Watson cuenta con información de los empleados de la empresa y algunos factores que pueden estar relacionados con la rotación del personal
### Variable objetivo: 
"Attrition" (Rotación) 

### Características: 
Age (Edad)
JobSatisfaction (Satisfacción con el trabajo)
EnvironmentSatisfaction (Satisfacción con el ambiente)
PerformanceRating (Desempeño)
YearsInCurrentRole (Años en el rol actual)
YearsAtCompany (Años en la empesa)
to.

.

## Preprocesamiento de datos
Se verifica que no exiten valores nulos, por lo que podemos continuar con la codificacion y escalado.
Seleccioanmos en un DataFRame las varibles que interesan para el modelo supervisado

In [62]:
#dataframe seleccionado
columnas_seleccionadas = ['Attrition', 'Age', 'JobSatisfaction', 'EnvironmentSatisfaction', 'PerformanceRating', 'YearsInCurrentRole', 'YearsAtCompany']
df_seleccionado = df[columnas_seleccionadas].copy()  # Hacer una copia para evitar el warning
df_seleccionado.head()

,Attrition,Age,JobSatisfaction,EnvironmentSatisfaction,PerformanceRating,YearsInCurrentRole,YearsAtCompany
0,Yes,41,4,2,3,4,6
1,No,49,2,3,4,7,10
2,Yes,37,3,4,3,0,0
3,No,33,3,4,3,7,8
4,No,27,2,1,3,2,2


### Convertir la variable dependiente Attrition a binaria

In [64]:
# Convertir 'Attrition' a binario (1 si 'Yes', 0 si 'No')
df_seleccionado['Attrition'] = df_seleccionado['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
df_seleccionado.head()

,Attrition,Age,JobSatisfaction,EnvironmentSatisfaction,PerformanceRating,YearsInCurrentRole,YearsAtCompany
0,1,41,4,2,3,4,6
1,0,49,2,3,4,7,10
2,1,37,3,4,3,0,0
3,0,33,3,4,3,7,8
4,0,27,2,1,3,2,2


In [29]:
# Asegurarse de que 'Attrition' sea de tipo int
df_seleccionado['Attrition'] = df_seleccionado['Attrition'].astype(int)

In [33]:
# Verificar los valores nulos
df_seleccionado.isnull().sum()
#No se registran valores nulos

Attrition                  0
Age                        0
JobSatisfaction            0
EnvironmentSatisfaction    0
PerformanceRating          0
YearsInCurrentRole         0
YearsAtCompany             0
dtype: int64

### Escalado de las variables numéricas:
Las variables 'Age', 'JobSatisfaction', 'EnvironmentSatisfaction', 'PerformanceRating', 'YearsInCurrentRole', 'YearsAtCompany'
debenn ser escaladas para que todas tengan el mismo rango y no haya sesgos hacia aquellas con valores más grandes.

In [35]:
# Escalar las características numéricas
scaler = StandardScaler()
df_seleccionado[['Age', 'JobSatisfaction', 'EnvironmentSatisfaction', 'PerformanceRating', 'YearsInCurrentRole', 'YearsAtCompany']] = scaler.fit_transform(
    df_seleccionado[['Age', 'JobSatisfaction', 'EnvironmentSatisfaction', 'PerformanceRating', 'YearsInCurrentRole', 'YearsAtCompany']]
)

### Dividir los datos en conjuntos de entrenamiento 80% y prueba 20%

In [37]:
#X = df_seleccionado[['Age', 'JobSatisfaction', 'EnvironmentSatisfaction', 'PerformanceRating', 'YearsInCurrentRole', 'YearsAtCompany']] # Características (features)
# Dividir los datos en entrenamiento y prueba
X = df_seleccionado.drop('Attrition', axis=1)
y = df_seleccionado['Attrition']

# Dividir en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Asegurarse de que X_train y y_train sean arrays de numpy
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()


### Entrenamietno el modelo de regresión logísticaa

In [39]:
# Crear y entrenar el modelo de regresión logística
modelo = LogisticRegression()
modelo.fit(X_train, y_train)


LogisticRegression()

In [45]:
# Hacer predicciones y evaluar el modelo
y_pred = modelo.predict(X_test)
print('\nReporte de clasificación:\n')
print(classification_report(y_test, y_pred))
print('Matriz de confusión:\n')
print(confusion_matrix(y_test, y_pred))



Reporte de clasificación:

              precision    recall  f1-score   support

           0       0.87      1.00      0.93       255
           1       0.00      0.00      0.00        39

    accuracy                           0.87       294
   macro avg       0.43      0.50      0.46       294
weighted avg       0.75      0.87      0.81       294

Matriz de confusión:

[[255   0]
 [ 39   0]]


C:\Users\UserP\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
C:\Users\UserP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\UserP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\UserP\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples

Se evidencia que es necesario balancear los datos ya que predice muy bien los que no cambian de rol pero no a los que si.

In [53]:
# Aplicar SMOTE
smote = SMOTE(sampling_strategy='minority')
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Dividir el conjunto balanceado en datos de entrenamiento y prueba
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Entrenar el modelo
from sklearn.linear_model import LogisticRegression
modelo = LogisticRegression()
modelo.fit(X_train_resampled, y_train_resampled)

# Evaluar el modelo
from sklearn.metrics import classification_report, confusion_matrix

y_pred_resampled = modelo.predict(X_test_resampled)
print(classification_report(y_test_resampled, y_pred_resampled))
print(confusion_matrix(y_test_resampled, y_pred_resampled))


              precision    recall  f1-score   support

           0       0.70      0.61      0.65       206
           1       0.62      0.72      0.67       186

    accuracy                           0.66       392
   macro avg       0.66      0.66      0.66       392
weighted avg       0.66      0.66      0.66       392

[[125  81]
 [ 53 133]]


El modelo de regresión logística ha identificado que un porcentaje significativo de empleados con alta probabilidad de rotación (cambio de rol) tiene un buen desempeño en cuanto a la capacidad de ser detectados, con un recall del 72% para esta clase. Sin embargo, también se observa que existe una tasa de falsos positivos, ya que algunos empleados que no cambiarán de rol fueron clasificados erróneamente como rotación. La clase mayoritaria (no rotación) está siendo predicha con mayor precisión (70%), pero con un recall menor (61%), lo que sugiere que el modelo tiene más dificultades para identificar correctamente a los empleados que no cambiarán de rol. En general, el modelo ha sido más efectivo en la identificación de empleados con alta probabilidad de rotación, pero aún hay margen de mejora para un equilibrio más efectivo entre ambas clases.